In [1]:
from elasticsearch import Elasticsearch

# Kết nối đến Elasticsearch
es = Elasticsearch([{'host': '10.2.65.106', 'port': 9200}])

# Kiểm tra kết nối
if es.ping():
    print("Kết nối thành công!")
else:
    print("Kết nối thất bại!")

# Tạo một index mới với dynamic mapping
index_name = 'wikitest'

# Kiểm tra xem index đã tồn tại chưa
if not es.indices.exists(index=index_name):
    # Tạo index nếu chưa tồn tại
    es.indices.create(index=index_name)
    print(f"Index '{index_name}' đã được tạo thành công!")
else:
    print(f"Index '{index_name}' đã tồn tại.")


Kết nối thành công!
Index 'wikitest' đã tồn tại.


In [2]:
import os 
import json 

folder = '2024-08-07'
for file in os.listdir(folder)[1:]:
    file = os.path.join(folder, file)
    data = json.load(open(file, 'r', encoding='utf8'))
    id = data['_id']
    del data['_id']
    es.index(index='wikitest', id=id, body=data) 



In [ ]:
# NOTE: Hàm embedding lại nội dung 


# NOTE: Tiến hành tìm kiếm 
# Thử nghiệm tìm kiếm chính xác, 
# Thử nghiệm tìm kiếm semantic 
# So sánh kết quả vơi Wikipedia 


In [3]:
# Tên của index bạn muốn query
index_name = 'wikitest'

# Query tất cả các bản ghi
query = {
    "query": {
        "match_all": {}
    }
}

# Thực hiện tìm kiếm với scroll
response = es.search(index=index_name, body=query, scroll='2m', size=1000)

# Lấy scroll ID
scroll_id = response['_scroll_id']
hits = response['hits']['hits']

dataset = []
# In kết quả ban đầu
for hit in hits:
    dataset.append(hit['_source'])

# Tiếp tục lấy dữ liệu cho đến khi không còn dữ liệu
while len(hits) > 0:
    response = es.scroll(scroll_id=scroll_id, scroll='2m')
    scroll_id = response['_scroll_id']
    hits = response['hits']['hits']
    for hit in hits:
        dataset.append(hit['_source'])

# Đóng kết nối
# es.transport.close()

len(dataset)

3900

In [4]:
import os 
os.environ['http_proxy'] = 'http://proxybsh.bkav.com:3128'
os.environ['https_proxy'] = 'http://proxybsh.bkav.com:3128'

from sentence_transformers import SentenceTransformer

# Tải mô hình vietnamese-sbert
model = SentenceTransformer('keepitreal/vietnamese-sbert')

# Chuỗi đầu vào
input_sentence = "Xin chào, tôi là trợ lý ảo AI."

# Mã hóa chuỗi đầu vào
import time 
begin = time.time()
embedding = model.encode(input_sentence)
print(time.time()-begin)
# In ra vector mã hóa
print("Embedding vector:", embedding)

os.environ['http_proxy'] = ''
os.environ['https_proxy'] = ''


c:\Users\SonTDc\miniconda3\envs\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\SonTDc\miniconda3\envs\venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SonTDc\.cache\huggingface\hub\models--keepitreal--vietnamese-sbert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

...

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding vector: [-0.35972357  0.11485553 -0.2999682   0.02430088 -0.06778071  0.3690547
  0.12980118  0.12190356 -0.37875205  0.15360476  0.01786158 -0.11528603
  0.00945097  0.2597296   0.33080503  0.13321061  0.27799985  0.08145962
 -0.6374745  -0.37308565  0.24443696 -0.5130566   0.6561708   0.01135225
 -0.00755878 -0.1373477   0.3573049   0.13722327 -0.5717716   0.20042038
 -0.03038551 -0.13771208  0.28654167 -0.09592875 -0.60999185  0.4753601
  0.22238903 -0.07319859  0.2873942   0.37282464 -0.15191753 -0.05534078
  0.15616393 -0.35953274  0.3785999   0.10582693  0.06377302  0.24590889
  0.36464828  0.14595903 -0.03964251  0.13896     0.38747743  0.39420053
  0.5699941   0.02002078 -0.00304876 -0.05180573 -0.20620649  0.27871686
 -0.02707338  0.112913   -0.41100207  0.23199095  0.06246358 -0.48498496
  0.15017875 -0.2728349   0.00864498 -0.04702333 -0.54909366  0.35681173
  0.7982814  -0.42214605 -0.15406856 -0.7150187   0.30748376 -0.08426168
  0.1520609   0.84043515  0.1011960

In [5]:
# Chuỗi đầu vào
input_sentence = "Xin chào, tôi là trợ lý ảo AI."

# Mã hóa chuỗi đầu vào
import time 
begin = time.time()
embedding = model.encode(input_sentence)
print(time.time()-begin)
# In ra vector mã hóa
print("Embedding vector:", embedding)

0.1570587158203125
Embedding vector: [-0.35972357  0.11485553 -0.2999682   0.02430088 -0.06778071  0.3690547
  0.12980118  0.12190356 -0.37875205  0.15360476  0.01786158 -0.11528603
  0.00945097  0.2597296   0.33080503  0.13321061  0.27799985  0.08145962
 -0.6374745  -0.37308565  0.24443696 -0.5130566   0.6561708   0.01135225
 -0.00755878 -0.1373477   0.3573049   0.13722327 -0.5717716   0.20042038
 -0.03038551 -0.13771208  0.28654167 -0.09592875 -0.60999185  0.4753601
  0.22238903 -0.07319859  0.2873942   0.37282464 -0.15191753 -0.05534078
  0.15616393 -0.35953274  0.3785999   0.10582693  0.06377302  0.24590889
  0.36464828  0.14595903 -0.03964251  0.13896     0.38747743  0.39420053
  0.5699941   0.02002078 -0.00304876 -0.05180573 -0.20620649  0.27871686
 -0.02707338  0.112913   -0.41100207  0.23199095  0.06246358 -0.48498496
  0.15017875 -0.2728349   0.00864498 -0.04702333 -0.54909366  0.35681173
  0.7982814  -0.42214605 -0.15406856 -0.7150187   0.30748376 -0.08426168
  0.1520609   0.

In [ ]:
def chunk_document():
    pass 


In [43]:
# ID của tài liệu bạn muốn cập nhật
doc_id = '1'

# Chỉ số và kiểu tài liệu
index_name = 'my_index'
doc_type = '_doc'

# Dữ liệu cập nhật
update_body = {
    "doc": {
        "field1": "new_value1",
        "field2": "new_value2"
    }
}

try:
    # Thực hiện cập nhật tài liệu
    response = es.update(index=index_name, id=doc_id, body=update_body, doc_type=doc_type)
    print("Tài liệu đã được cập nhật:", response)
except Exception as e:
    print("Có lỗi xảy ra:", e)


(13656) 1997 EX45 lÃ  má»t tiá»u hÃ nh tinh vÃ nh Äai chÃ­nh. NÃ³ ÄÆ°á»£c phÃ¡t hiá»n bá»i ChÆ°Æ¡ng trÃ¬nh tiá»u hÃ nh tinh Báº¯c Kinh Schmidt CCD á» tráº¡m Xinglong á» tá»nh Há» Báº¯c, Trung Quá»c ngÃ y 15 thÃ¡ng 3 nÄm 1997.[1]

Xem thÃªm[sá»­a | sá»­a mÃ£ nguá»n]
Danh sÃ¡ch cÃ¡c tiá»u hÃ nh tinh: 13001â14000
Tham kháº£o[sá»­a | sá»­a mÃ£ nguá»n]


^ JPL Small-Body Database Browser


xtsÄá»nh vá» tiá»u hÃ nh tinh
(13655) 1997 ER2
(13656) 1997 EX45
13657 Badinter

xtsCÃ¡c thiÃªn thá» nhá» trong há» Máº·t Trá»iHÃ nh tinh vi hÃ¬nh
Äá»nh danh
NhÃ³m
Danh sÃ¡ch
Vá» tinh
NghÄ©a cá»§a tÃªn
PhÃ¡t Ã¢m tÃªn
Tiá»u hÃ nh tinh
Tiá»u hÃ nh tinh Aten
CÃ¡c gia ÄÃ¬nh
ThiÃªn thá» Troia cá»§a Sao Má»c
VÃ nh Äai tiá»u hÃ nh tinh
ThiÃªn thá» gáº§n TrÃ¡i Äáº¥t
Loáº¡i quang phá»
HÃ nh tinh nhá» xa
Centaur
Damocloid
ThiÃªn thá» Troia cá»§a Sao Háº£i VÆ°Æ¡ng
ThiÃªn thá» bÃªn ngoÃ i Sao Háº£i VÆ°Æ¡ng
ThiÃªn thá» tÃ¡ch ra
VÃ nh Äai Kuiper
ÄÃ¡m mÃ¢y Oort
ÄÄ©a phÃ¢n t

In [46]:
encoded_str = "má»t tiá»u hÃ nh tinh vÃ nh Äai chÃ­nh"
decoded_str = bytes(encoded_str, "utf8").decode("unicode_escape")
print(decoded_str)

mÃ¡Â»Ât tiÃ¡Â»Âu hÃ nh tinh vÃ nh ÃÂai chÃÂ­nh


In [55]:
import os 
import json 

folder = '2024-08-07'
for file in os.listdir(folder)[1:]:
    file = os.path.join(folder, file)
    data = json.loads(open(file, 'rb').read().decode('utf8'))
    decoded_string = data
    print(decoded_string)
    break 

{'Bot_id': 0, '_id': '77bc5e2c3cebce2ccbc82d45a50ca18bec39c6c093be5fa5d542f13b3d35b873', 'type': 'wikipedia', 'title': '(7982) 1979 MX5', 'link': 'https://vi.wikipedia.org//wiki/(7982)_1979_MX5', 'crawled_time': '2024/08/07 11:01:56', 'content': '\\n(7982) 1979 MX5 l\\xc3\\xa0 m\\xe1\\xbb\\x99t ti\\xe1\\xbb\\x83u h\\xc3\\xa0nh tinh v\\xc3\\xa0nh \\xc4\\x91ai ch\\xc3\\xadnh. N\\xc3\\xb3 \\xc4\\x91\\xc6\\xb0\\xe1\\xbb\\xa3c ph\\xc3\\xa1t hi\\xe1\\xbb\\x87n b\\xe1\\xbb\\x9fi Eleanor F. Helin v\\xc3\\xa0 Schelte J. Bus \\xe1\\xbb\\x9f \\xc4\\x90\\xc3\\xa0i thi\\xc3\\xaan v\\xc4\\x83n Siding Spring g\\xe1\\xba\\xa7n Coonabarabran, New South Wales, \\xc3\\x9ac, ng\\xc3\\xa0y 25 th\\xc3\\xa1ng 6 n\\xc4\\x83m 1979.\\n\\nXem th\\xc3\\xaam[s\\xe1\\xbb\\xada | s\\xe1\\xbb\\xada m\\xc3\\xa3 ngu\\xe1\\xbb\\x93n]\\nDanh s\\xc3\\xa1ch c\\xc3\\xa1c ti\\xe1\\xbb\\x83u h\\xc3\\xa0nh tinh: 7001–8000\\nTham kh\\xe1\\xba\\xa3o[s\\xe1\\xbb\\xada | s\\xe1\\xbb\\xada m\\xc3\\xa3 ngu\\xe1\\xbb\\x93n]\\n\\n\\nL

In [26]:
!curl -X PUT "localhost:9200/wikitest/_settings" -H 'Content-Type: application/json' -d'{"index.blocks.read_only_allow_delete": null}'

{"acknowledged":true}

In [ ]:
# Đóng kết nối
es.transport.close()